In [42]:
import pandas as pd
import yfinance as yf
import eventstudy as es
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import math 
from numpy import NaN

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from scipy import stats
from scipy.stats import kurtosis, skew

%matplotlib inline

<b> Below is all the code used in the final project. The first part is the reverse stock split analysis. The second part is the S\&P vs NASDAQ analysis. Tables for the two analyses are on the paper. Scroll down to the end for bonus event studies on Pfizer and Apple! <b>

### Project Regression 1 
### Aggregating all reverse stock splits for one regression
For this regression, event windows will not be used. I will simply determine if abnormal returns are significantly different from 0 for the announcement day and for the split day. 

In [3]:
sandp = yf.Ticker('^GSPC')
sandphist = sandp.history(start='2019-03-23', end ='2020-12-4')
sandphist['returns'] = sandphist['Close'].pct_change(1)

In [4]:
# For Brookfield Asset Management (BAM)
bam = yf.Ticker('BAM')

bamhist = bam.history(start="2020-01-28", end="2020-12-03")
bamhist['returns'] = bamhist['Close'].pct_change(1)

mbam = sandphist["2020-01-28":"2020-12-03"]
bamhist['S&P Returns'] = mbam['returns']

bamhist['Event'] = 0
bamhist['Split'] = 0
bamhist.loc['2020-02-13', 'Event'] = 1
bamhist.loc['2020-04-02','Split'] = 1
#print(bamhist.to_string())
bamreg = bamhist[['returns', 'S&P Returns', 'Event', 'Split']]

In [5]:
# For Nabors Industries 
nbr = yf.Ticker('NBR')

nbrhist = nbr.history(start="2020-03-23", end="2020-12-03")
nbrhist['returns'] = nbrhist['Close'].pct_change(1)

mnbr = sandphist["2020-03-23":"2020-12-03"]
nbrhist['S&P Returns'] = mnbr['returns']

nbrhist['Event'] = 0
nbrhist['Split'] = 0
nbrhist.loc['2020-04-21', 'Event'] = 1
nbrhist.loc['2020-04-23','Split'] = 1
nbrreg = nbrhist[['returns', 'S&P Returns', 'Event', 'Split']]
#print(nbrreg.to_string())

In [6]:
# For Scorpio Bulkers
salt = yf.Ticker('SALT')

salthist = salt.history(start="2020-03-23", end="2020-12-03")
salthist['returns'] = salthist['Close'].pct_change(1)

msalt = sandphist["2020-03-23":"2020-12-03"]
salthist['S&P Returns'] = msalt['returns']

salthist['Event'] = 0
salthist['Split'] = 0
salthist.loc['2020-04-03', 'Event'] = 1
salthist.loc['2020-04-07','Split'] = 1
saltreg = nbrhist[['returns', 'S&P Returns', 'Event', 'Split']]
#print(nbrreg.to_string())

In [7]:
# For Aurora Cannabis
acb = yf.Ticker('ACB')

acbhist = acb.history(start="2020-03-23", end="2020-12-03")
acbhist['returns'] = acbhist['Close'].pct_change(1)

macb = sandphist["2020-03-23":"2020-12-03"]
acbhist['S&P Returns'] = macb['returns']

acbhist['Event'] = 0
acbhist['Split'] = 0
acbhist.loc['2020-04-13', 'Event'] = 1
acbhist.loc['2020-05-11','Split'] = 1
acbreg = acbhist[['returns', 'S&P Returns', 'Event', 'Split']]
#print(acbreg.to_string())

In [8]:
# For DSS 
dss = yf.Ticker('DSS')

dsshist = dss.history(start="2020-03-23", end="2020-12-03")
dsshist['returns'] = acbhist['Close'].pct_change(1)

mdss = sandphist["2020-03-23":"2020-12-03"]
dsshist['S&P Returns'] = mdss['returns']

dsshist['Event'] = 0
dsshist['Split'] = 0
dsshist.loc['2020-05-06', 'Event'] = 1
dsshist.loc['2020-05-08','Split'] = 1
dssreg = dsshist[['returns', 'S&P Returns', 'Event', 'Split']]
#print(dssreg.to_string())

In [9]:
# For Timber Pharmaceuticals
rand = yf.Ticker('TMBR')

randhist = rand.history(start="2020-03-23", end="2020-12-03")
randhist['returns'] = randhist['Close'].pct_change(1)

mrand = sandphist["2020-03-23":"2020-12-03"]
randhist['S&P Returns'] = mrand['returns']

randhist['Event'] = 0
randhist['Split'] = 0
randhist.loc['2020-05-18', 'Event'] = 1
randhist.loc['2020-05-19','Split'] = 1
tmbrreg = randhist[['returns', 'S&P Returns', 'Event', 'Split']]
#print(randreg.to_string())

In [10]:
# For Avita Therapeutics
rcel = yf.Ticker('RCEL')

rcelhist = rcel.history(start="2020-03-23", end="2020-12-03")
rcelhist['returns'] = rcelhist['Close'].pct_change(1)

mrcel = sandphist["2020-03-23":"2020-12-03"]
rcelhist['S&P Returns'] = mrcel['returns']

rcelhist['Event'] = 0
rcelhist['Split'] = 0
rcelhist.loc['2020-06-17', 'Event'] = 1
rcelhist.loc['2020-06-30','Split'] = 1
rcelreg = rcelhist[['returns', 'S&P Returns', 'Event', 'Split']]
#print(rcelreg.to_string())

In [11]:
# For Energy Focus Inc 
efoi = yf.Ticker('EFOI')

efoihist = efoi.history(start="2020-03-23", end="2020-12-03")
efoihist['returns'] = efoihist['Close'].pct_change(1)

mefoi = sandphist["2020-03-23":"2020-12-03"]
efoihist['S&P Returns'] = mefoi['returns']

efoihist['Event'] = 0
efoihist['Split'] = 0
efoihist.loc['2020-06-09', 'Event'] = 1
efoihist.loc['2020-06-12','Split'] = 1
efoireg = efoihist[['returns', 'S&P Returns', 'Event', 'Split']]
#print(efoireg.to_string())

In [12]:
# For FS KKR Group
fsk = yf.Ticker('FSK')

fskhist = fsk.history(start="2020-03-23", end="2020-12-03")
fskhist['returns'] = fskhist['Close'].pct_change(1)

mfsk = sandphist["2020-03-23":"2020-12-03"]
fskhist['S&P Returns'] = mfsk['returns']

fskhist['Event'] = 0
fskhist['Split'] = 0
fskhist.loc['2020-06-08', 'Event'] = 1
fskhist.loc['2020-06-16','Split'] = 1
fskreg = fskhist[['returns', 'S&P Returns', 'Event', 'Split']]
#print(fskreg.to_string())

In [13]:
# For XpresSpa Group
xspa = yf.Ticker('XSPA')

xspahist = xspa.history(start="2020-03-23", end="2020-12-03")
xspahist['returns'] = xspahist['Close'].pct_change(1)

mxspa = sandphist["2020-03-23":"2020-12-03"]
xspahist['S&P Returns'] = mxspa['returns']

xspahist['Event'] = 0
xspahist['Split'] = 0
xspahist.loc['2020-06-10', 'Event'] = 1
xspahist.loc['2020-06-11','Split'] = 1
xspareg = xspahist[['returns', 'S&P Returns', 'Event', 'Split']]
#print(xspareg.to_string())

In [14]:
# For Ashford Hospitality Trust 
aht = yf.Ticker('AHT')

ahthist = aht.history(start="2020-03-23", end="2020-12-03")
ahthist['returns'] = ahthist['Close'].pct_change(1)

maht = sandphist["2020-03-23":"2020-12-03"]
ahthist['S&P Returns'] = maht['returns']

ahthist['Event'] = 0
ahthist['Split'] = 0
ahthist.loc['2020-07-02', 'Event'] = 1
ahthist.loc['2020-07-17','Split'] = 1
ahtreg = ahthist[['returns', 'S&P Returns', 'Event', 'Split']]
#print(ahtreg.to_string())

In [15]:
# For ODP Corp 
odp = yf.Ticker('ODP')

odphist = odp.history(start="2020-03-23", end="2020-12-03")
odphist['returns'] = odphist['Close'].pct_change(1)

modp = sandphist["2020-03-23":"2020-12-03"]
odphist['S&P Returns'] = modp['returns']

odphist['Event'] = 0
odphist['Split'] = 0
odphist.loc['2020-06-26', 'Event'] = 1
odphist.loc['2020-07-01','Split'] = 1
odpreg = odphist[['returns', 'S&P Returns', 'Event', 'Split']]
#print(odpreg.to_string())

In [16]:
# For SCYNEXIS Inc 
scyx = yf.Ticker('SCYX')

scyxhist = scyx.history(start="2020-03-23", end="2020-12-03")
scyxhist['returns'] = scyxhist['Close'].pct_change(1)

mscyx = sandphist["2020-03-23":"2020-12-03"]
scyxhist['S&P Returns'] = mscyx['returns']

scyxhist['Event'] = 0
scyxhist['Split'] = 0
scyxhist.loc['2020-06-26', 'Event'] = 1
scyxhist.loc['2020-07-01','Split'] = 1
scyxreg = scyxhist[['returns', 'S&P Returns', 'Event', 'Split']]
#print(scyxreg.to_string())

In [17]:
# For JAKK Inc 
jakk = yf.Ticker('JAKK')

jakkhist = jakk.history(start="2020-03-23", end="2020-12-03")
jakkhist['returns'] = jakkhist['Close'].pct_change(1)

mjakk = sandphist["2020-03-23":"2020-12-03"]
jakkhist['S&P Returns'] = mjakk['returns']

jakkhist['Event'] = 0
jakkhist['Split'] = 0
jakkhist.loc['2020-07-09', 'Event'] = 1
jakkhist.loc['2020-07-10','Split'] = 1
jakkreg = jakkhist[['returns', 'S&P Returns', 'Event', 'Split']]
#print(jakkreg.to_string())

In [18]:
# For Actinium Pharmaceuticals Inc 
atnm = yf.Ticker('ATNM')

atnmhist = atnm.history(start="2020-03-23", end="2020-12-03")
atnmhist['returns'] = atnmhist['Close'].pct_change(1)

matnm = sandphist["2020-03-23":"2020-12-03"]
atnmhist['S&P Returns'] = matnm['returns']

atnmhist['Event'] = 0
atnmhist['Split'] = 0
atnmhist.loc['2020-08-10', 'Event'] = 1
atnmhist.loc['2020-08-11','Split'] = 1
atnmreg = atnmhist[['returns', 'S&P Returns', 'Event', 'Split']]
#print(atnmreg.to_string())

In [19]:
# For Apple 
aapl = yf.Ticker('AAPL')

aaplhist = aapl.history(start="2020-03-23", end="2020-12-03")
aaplhist['returns'] = aaplhist['Close'].pct_change(1)

maapl = sandphist["2020-03-23":"2020-12-03"]
aaplhist['S&P Returns'] = maapl['returns']

aaplhist['Event'] = 0
aaplhist['Split'] = 0
aaplhist.loc['2020-07-31', 'Event'] = 1
aaplhist.loc['2020-08-31','Split'] = 1
aaplreg = aaplhist[['returns', 'S&P Returns', 'Event', 'Split']]
#print(aaplreg.to_string())

In [20]:
# For Tesla 
tsla = yf.Ticker('TSLA')

tslahist = tsla.history(start="2020-03-23", end="2020-12-03")
tslahist['returns'] = tslahist['Close'].pct_change(1)

mtsla = sandphist["2020-03-23":"2020-12-03"]
tslahist['S&P Returns'] = mtsla['returns']

tslahist['Event'] = 0
tslahist['Split'] = 0
tslahist.loc['2020-08-12', 'Event'] = 1
tslahist.loc['2020-08-31','Split'] = 1
tslareg = tslahist[['returns', 'S&P Returns', 'Event', 'Split']]
#print(tslareg.to_string())

In [21]:
# For TOP SHIPS Inc
tops = yf.Ticker('TOPS')

topshist = tops.history(start="2020-03-23", end="2020-12-03")
topshist['returns'] = topshist['Close'].pct_change(1)

mtops = sandphist["2020-03-23":"2020-12-03"]
topshist['S&P Returns'] = mtops['returns']

topshist['Event'] = 0
topshist['Split'] = 0
topshist.loc['2020-08-07', 'Event'] = 1
topshist.loc['2020-08-10','Split'] = 1
topsreg = topshist[['returns', 'S&P Returns', 'Event', 'Split']]
#print(topsreg.to_string())

In [22]:
# For Biocept Inc
bioc = yf.Ticker('BIOC')

biochist = bioc.history(start="2020-03-23", end="2020-12-03")
biochist['returns'] = biochist['Close'].pct_change(1)

mbioc = sandphist["2020-03-23":"2020-12-03"]
biochist['S&P Returns'] = mbioc['returns']

biochist['Event'] = 0
biochist['Split'] = 0
biochist.loc['2020-09-04', 'Event'] = 1
biochist.loc['2020-09-08','Split'] = 1
biocreg = biochist[['returns', 'S&P Returns', 'Event', 'Split']]
#print(biocreg.to_string())

In [23]:
# For Humanigen Inc
hgen = yf.Ticker('HGEN')

hgenhist = hgen.history(start="2020-03-23", end="2020-12-03")
hgenhist['returns'] = hgenhist['Close'].pct_change(1)

mhgen = sandphist["2020-03-23":"2020-12-03"]
hgenhist['S&P Returns'] = mhgen['returns']

hgenhist['Event'] = 0
hgenhist['Split'] = 0
hgenhist.loc['2020-09-08', 'Event'] = 1
hgenhist.loc['2020-09-14','Split'] = 1
hgenreg = hgenhist[['returns', 'S&P Returns', 'Event', 'Split']]
#print(hgenreg.to_string())

In [24]:
# For Trex Company Inc
trex = yf.Ticker('TREX')

trexhist = trex.history(start="2020-03-23", end="2020-12-03")
trexhist['returns'] = trexhist['Close'].pct_change(1)

mtrex = sandphist["2020-03-23":"2020-12-03"]
trexhist['S&P Returns'] = mtrex['returns']

trexhist['Event'] = 0
trexhist['Split'] = 0
trexhist.loc['2020-08-03', 'Event'] = 1
trexhist.loc['2020-09-15','Split'] = 1
trexreg = trexhist[['returns', 'S&P Returns', 'Event', 'Split']]
#print(trexreg.to_string())

In [25]:
# For NetEase Inc
ntes = yf.Ticker('NTES')

nteshist = ntes.history(start="2020-03-23", end="2020-12-03")
nteshist['returns'] = nteshist['Close'].pct_change(1)

mntes = sandphist["2020-03-23":"2020-12-03"]
nteshist['S&P Returns'] = mntes['returns']

nteshist['Event'] = 0
nteshist['Split'] = 0
nteshist.loc['2020-09-21', 'Event'] = 1
nteshist.loc['2020-10-02','Split'] = 1
ntesreg = nteshist[['returns', 'S&P Returns', 'Event', 'Split']]
#print(ntesreg.to_string())

In [26]:
# For NextEra Energy Inc
nee = yf.Ticker('NEE')

neehist = nee.history(start="2020-03-23", end="2020-12-03")
neehist['returns'] = neehist['Close'].pct_change(1)

mnee = sandphist["2020-03-23":"2020-12-03"]
neehist['S&P Returns'] = mnee['returns']

neehist['Event'] = 0
neehist['Split'] = 0
neehist.loc['2020-09-15', 'Event'] = 1
neehist.loc['2020-10-27','Split'] = 1
neereg = neehist[['returns', 'S&P Returns', 'Event', 'Split']]
#print(ntesreg.to_string())

In [27]:
# Combine the 21 stock splits into one dataframe and run the regression
bigreg = pd.concat([neereg, ntesreg, trexreg,hgenreg,topsreg,biocreg, atnmreg,jakkreg, scyxreg,odpreg,xspareg,fskreg,
          efoireg,rcelreg,tmbrreg,dssreg,acbreg,bamreg,saltreg,nbrreg])
bigreg.rename(columns = {'Event':'Announcement'}, inplace = True)
X = bigreg.drop('returns', axis = 1)
Y = bigreg[['returns']]

model = sm.OLS(Y, X, missing='drop')
result = model.fit()
#result.summary()

## Project Regression 2
### Announcements with technology stocks
Below, I analyze tech stocks, the most notable sector of recent months.

### AMD's stock on Intel's announcement
On the same day as Intel's announcement, AMD's (AMD) prices shot up so that it exceeded the price of Intel's shares. Investors also incorporated information of Intel's failure into AMD's stock prices. I use an event window of 10 trading days. 

In [28]:
amd = yf.Ticker('AMD')

amdhist = amd.history(start="2020-03-23", end="2020-12-03")
amdhist['returns'] = amdhist['Close'].pct_change(1)

mamd = sandphist["2020-03-23":"2020-12-03"]
amdhist['S&P Returns'] = mamd['returns']

# Run the first regression on the day of the announcement
amdhist['Event'] = 0
amdhist['Anticipation'] = 0
amdhist['Adjustment'] = 0
amdhist.loc['2020-07-24', 'Event'] = 1
amdhist.loc['2020-07-10':'2020-07-23','Anticipation'] = 1
amdhist.loc['2020-07-25':'2020-08-07','Adjustment'] = 1
#print(amdhist.to_string())
amdreg = amdhist[['returns', 'S&P Returns', 'Event', 'Anticipation', 'Adjustment']]

### Microsoft announces dividend increase
On September 15, Microsoft (MSFT) announced a dividend increase. The market hardly reacted to this information

In [29]:
msft = yf.Ticker('MSFT')

msfthist = msft.history(start="2020-03-23", end="2020-12-03")
msfthist['returns'] = msfthist['Close'].pct_change(1)

mmsft = sandphist["2020-03-23":"2020-12-03"]
msfthist['S&P Returns'] = mmsft['returns']

# Run the first regression on the day of the announcement
msfthist['Event'] = 0
msfthist['Anticipation'] = 0
msfthist['Adjustment'] = 0
msfthist.loc['2020-09-15', 'Event'] = 1
msfthist.loc['2020-09-01':'2020-09-14','Anticipation'] = 1
msfthist.loc['2020-09-16':'2020-09-29','Adjustment'] = 1
#print(msfthist.to_string())
msftreg = msfthist[['returns', 'S&P Returns', 'Event', 'Anticipation', 'Adjustment']]

### Texas Instruments announced dividend increase
On 9/21, Texas instruments announced a dividend increase. As with Microsoft, the market didn't reach much to the news

In [30]:
txn = yf.Ticker('TXN')

txnhist = txn.history(start="2020-03-23", end="2020-12-03")
txnhist['returns'] = txnhist['Close'].pct_change(1)

mtxn = sandphist["2020-03-23":"2020-12-03"]
txnhist['S&P Returns'] = mtxn['returns']

# Run the first regression on the day of the announcement
txnhist['Event'] = 0
txnhist['Anticipation'] = 0
txnhist['Adjustment'] = 0
txnhist.loc['2020-09-21', 'Event'] = 1
txnhist.loc['2020-09-07':'2020-09-18','Anticipation'] = 1
txnhist.loc['2020-09-22':'2020-10-05','Adjustment'] = 1
#print(txnhist.to_string())
txnreg = txnhist[['returns', 'S&P Returns', 'Event', 'Anticipation', 'Adjustment']]

### Tesla stock hits \$1000 
On 6/10, Tesla stock hit $1000. I use event windows of 10 days. Conclusions show high reaction to the news and some time to adjust.

In [31]:
tsla = yf.Ticker('TSLA')

tslahist = tsla.history(start="2020-03-23", end="2020-09-30")
tslahist['returns'] = tslahist['Close'].pct_change(1)

mtsla = sandphist["2020-03-23":"2020-09-30"]
tslahist['S&P Returns'] = mtsla['returns']

# Run the first regression on the day of the announcement
tslahist['Event'] = 0
tslahist['Anticipation'] = 0
tslahist['Adjustment'] = 0
tslahist.loc['2020-06-10', 'Event'] = 1
tslahist.loc['2020-05-26':'2020-06-09','Anticipation'] = 1
tslahist.loc['2020-06-11':'2020-06-24','Adjustment'] = 1
#print(tslahist.to_string())
tslareg = tslahist[['returns', 'S&P Returns', 'Event', 'Anticipation', 'Adjustment']]

In [32]:
# combining them into one dataframe and running regression
sandpreg = pd.concat([amdreg, msftreg, txnreg, tslareg])
X = sandpreg.drop('returns', axis = 1)
Y = sandpreg[['returns']]

model = sm.OLS(Y, X, missing='drop')
result = model.fit()
#result.summary()

## Regressing on NASDAQ 
The S&P 500 may not be a suitable control for the 4 stocks above. Of all the sectors, technology has risen the most, so I regress on the returns of the NASDAQ composite and see if that makes any difference in conclusions.

In [34]:
#Fetching returns of NASDAQ composite over time frame
nsdq = yf.Ticker('^IXIC')
nsdqhist = nsdq.history(start='2019-01-01', end ='2020-12-4')
nsdqhist['returns'] = nsdqhist['Close'].pct_change(1)
#nsdqhist.head()

In [35]:
amd = yf.Ticker('AMD')

amdhist = amd.history(start="2020-03-23", end="2020-12-03")
amdhist['returns'] = amdhist['Close'].pct_change(1)

mamd = nsdqhist["2020-03-23":"2020-12-03"]
amdhist['NSDQ Returns'] = mamd['returns']

# Run the first regression on the day of the announcement
amdhist['Event'] = 0
amdhist['Anticipation'] = 0
amdhist['Adjustment'] = 0
amdhist.loc['2020-07-24', 'Event'] = 1
amdhist.loc['2020-07-10':'2020-07-23','Anticipation'] = 1
amdhist.loc['2020-07-25':'2020-08-07','Adjustment'] = 1
#print(amdhist.to_string())
amdreg = amdhist[['returns', 'NSDQ Returns', 'Event', 'Anticipation', 'Adjustment']]

msft = yf.Ticker('MSFT')

msfthist = msft.history(start="2020-03-23", end="2020-12-03")
msfthist['returns'] = msfthist['Close'].pct_change(1)

mmsft = nsdqhist["2020-03-23":"2020-12-03"]
msfthist['NSDQ Returns'] = mmsft['returns']

# Run the first regression on the day of the announcement
msfthist['Event'] = 0
msfthist['Anticipation'] = 0
msfthist['Adjustment'] = 0
msfthist.loc['2020-09-15', 'Event'] = 1
msfthist.loc['2020-09-01':'2020-09-14','Anticipation'] = 1
msfthist.loc['2020-09-16':'2020-09-29','Adjustment'] = 1
#print(msfthist.to_string())
msftreg = msfthist[['returns', 'NSDQ Returns', 'Event', 'Anticipation', 'Adjustment']]

txn = yf.Ticker('TXN')

txnhist = txn.history(start="2020-03-23", end="2020-12-03")
txnhist['returns'] = txnhist['Close'].pct_change(1)

mtxn = nsdqhist["2020-03-23":"2020-12-03"]
txnhist['NSDQ Returns'] = mtxn['returns']

# Run the first regression on the day of the announcement
txnhist['Event'] = 0
txnhist['Anticipation'] = 0
txnhist['Adjustment'] = 0
txnhist.loc['2020-09-21', 'Event'] = 1
txnhist.loc['2020-09-07':'2020-09-18','Anticipation'] = 1
txnhist.loc['2020-09-22':'2020-10-05','Adjustment'] = 1
#print(txnhist.to_string())
txnreg = txnhist[['returns', 'NSDQ Returns', 'Event', 'Anticipation', 'Adjustment']]

tsla = yf.Ticker('TSLA')

tslahist = tsla.history(start="2020-03-23", end="2020-09-30")
tslahist['returns'] = tslahist['Close'].pct_change(1)

mtsla = nsdqhist["2020-03-23":"2020-09-30"]
tslahist['NSDQ Returns'] = mtsla['returns']

# Run the first regression on the day of the announcement
tslahist['Event'] = 0
tslahist['Anticipation'] = 0
tslahist['Adjustment'] = 0
tslahist.loc['2020-06-10', 'Event'] = 1
tslahist.loc['2020-05-27':'2020-06-09','Anticipation'] = 1
tslahist.loc['2020-06-11':'2020-06-27','Adjustment'] = 1
#print(tslahist.to_string())
tslareg = tslahist[['returns', 'NSDQ Returns', 'Event', 'Anticipation', 'Adjustment']]

In [37]:
# combining them into one dataframe and running regression
nsdqreg = pd.concat([amdreg, msftreg, txnreg, tslareg])
X = nsdqreg.drop('returns', axis = 1)
Y = nsdqreg[['returns']]

model = sm.OLS(Y, X, missing='drop')
result = model.fit()
#result.summary()

## Bonus Stuff 

### Pfizer announces vaccine efficacy
On the morning of 11/9, Pfizer (PFE) announced a vaccine candidate with more than 90% efficacy. How fast did the market price in this news? I use an anticipation time window of 10 trading days and an adjustment time window of 10 trading days. Results show the market reacted quite fast (adjustment window significance is low) as vaccine news is monitored closely

In [38]:
pfe = yf.Ticker('PFE')

pfehist = pfe.history(start="2020-10-24", end="2020-11-25")
pfehist['returns'] = pfehist['Close'].pct_change(1)

mpfe = sandphist["2020-10-24":"2020-11-25"]
pfehist['S&P Returns'] = mpfe['returns']

# Run the first regression on the day of the announcement
pfehist['Event'] = 0
pfehist['Anticipation'] = 0
pfehist['Adjustment'] = 0
pfehist.loc['2020-11-09', 'Event'] = 1
pfehist.loc['2020-10-26':'2020-11-06','Anticipation'] = 1
pfehist.loc['2020-11-10':'2020-11-23','Adjustment'] = 1
pfereg = pfehist[['returns', 'S&P Returns', 'Event', 'Anticipation', 'Adjustment']]
#print(pfereg.to_string())

X = pfereg.drop('returns', axis = 1)
Y = pfereg[['returns']]

model = sm.OLS(Y, X, missing='drop')
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                returns   R-squared (uncentered):                   0.551
Model:                            OLS   Adj. R-squared (uncentered):              0.445
Method:                 Least Squares   F-statistic:                              5.215
Date:                Thu, 10 Dec 2020   Prob (F-statistic):                     0.00629
Time:                        17:20:18   Log-Likelihood:                          55.435
No. Observations:                  21   AIC:                                     -102.9
Df Residuals:                      17   BIC:                                     -98.69
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
S&P Returns      0.6646      0.314      2.118      0.049       0.003       1.327
Event            0.0691      0.020      3.538      0.003       0.028       0.110
Anticipation    -0.0056      0.007     -0.857      0.404      -0.019       0.008
Adjustment      -0.0022      0.006     -0.359      0.724      -0.015       0.011
==============================================================================
Omnibus:                        0.532   Durbin-Watson:                   2.666
Prob(Omnibus):                  0.766   Jarque-Bera (JB):                0.622
Skew:                          -0.205   Prob(JB):                        0.733
Kurtosis:                       2.263   Cond. No.                         51.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Apple Stock Split
Apple (AAPL) announced a stock split on 7/30 with the actual split occurring on 8/31. We run two regressions for both event dates. We make our anticipation window 10 trading days before the event days (7/30 and 8/31) and 10 trading days after those event days. There is high significance for both the announcement and the split dates but significance on the split date may be more likely due to the lackluster announcement of the iPhone 12 

In [39]:
aapl = yf.Ticker('AAPL')

aaplhist = aapl.history(start="2020-03-23", end="2020-09-30")
aaplhist['returns'] = aaplhist['Close'].pct_change(1)

mappl = sandphist["2020-03-23":"2020-09-30"]
aaplhist['S&P Returns'] = mappl['returns']

#Run the first regression on the day of the announcement. We use 7/31 as the event date since the announcement was made 
#after the market closed

aaplhist['Event'] = 0
aaplhist['Anticipation'] = 0
aaplhist['Adjustment'] = 0
aaplhist.loc['2020-07-31', 'Event'] = 1
aaplhist.loc['2020-07-16':'2020-07-30','Anticipation'] = 1
aaplhist.loc['2020-08-01':'2020-08-13','Adjustment'] = 1
#print(aaplhist.to_string())
aaplreg = aaplhist[['returns', 'S&P Returns', 'Event', 'Anticipation', 'Adjustment']]

X = aaplreg.drop('returns', axis = 1)
Y = aaplreg[['returns']]

model = sm.OLS(Y, X, missing='drop')
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                returns   R-squared (uncentered):                   0.669
Model:                            OLS   Adj. R-squared (uncentered):              0.659
Method:                 Least Squares   F-statistic:                              64.65
Date:                Thu, 10 Dec 2020   Prob (F-statistic):                    8.30e-30
Time:                        17:21:50   Log-Likelihood:                          360.78
No. Observations:                 132   AIC:                                     -713.6
Df Residuals:                     128   BIC:                                     -702.0
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
S&P Returns      1.0636      0.073     14.579      0.000       0.919       1.208
Event            0.0965      0.016      6.039      0.000       0.065       0.128
Anticipation    -0.0019      0.005     -0.386      0.700      -0.011       0.008
Adjustment       0.0056      0.005      1.051      0.295      -0.005       0.016
==============================================================================
Omnibus:                        4.347   Durbin-Watson:                   1.821
Prob(Omnibus):                  0.114   Jarque-Bera (JB):                5.678
Skew:                          -0.019   Prob(JB):                       0.0585
Kurtosis:                       4.015   Cond. No.                         15.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [40]:
# Run the second regression on the day of the split (8/31) and compare with the first 

aaplhist['Event'] = 0
aaplhist['Anticipation'] = 0
aaplhist['Adjustment'] = 0
aaplhist.loc['2020-08-31', 'Event'] = 1
aaplhist.loc['2020-08-10':'2020-08-30','Anticipation'] = 1
aaplhist.loc['2020-09-01':'2020-09-14','Adjustment'] = 1
aaplreg = aaplhist[['returns', 'S&P Returns', 'Event', 'Anticipation', 'Adjustment']]

In [41]:
X = aaplreg.drop('returns', axis = 1)
Y = aaplreg[['returns']]

model = sm.OLS(Y, X, missing='drop')
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                returns   R-squared (uncentered):                   0.593
Model:                            OLS   Adj. R-squared (uncentered):              0.581
Method:                 Least Squares   F-statistic:                              46.68
Date:                Thu, 10 Dec 2020   Prob (F-statistic):                    3.83e-24
Time:                        17:22:08   Log-Likelihood:                          347.20
No. Observations:                 132   AIC:                                     -686.4
Df Residuals:                     128   BIC:                                     -674.9
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
S&P Returns      1.0744      0.081     13.276      0.000       0.914       1.235
Event            0.0363      0.018      2.048      0.043       0.001       0.071
Anticipation     0.0047      0.005      1.018      0.311      -0.004       0.014
Adjustment      -0.0076      0.006     -1.293      0.198      -0.019       0.004
==============================================================================
Omnibus:                       50.013   Durbin-Watson:                   1.783
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              224.217
Skew:                           1.246   Prob(JB):                     2.05e-49
Kurtosis:                       8.878   Cond. No.                         17.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""